In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from __future__ import absolute_import, division, print_function

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Dense, Flatten, Conv2D, Activation
from tensorflow.keras import Model


In [0]:
import os,sys,inspect
import os
import joblib
import tensorflow as tf
import numpy as np
import h5py
import scipy.sparse.linalg as la
import scipy.sparse as sp
import scipy
import time
import pickle

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

import scipy.io as sio
# import process_data

from google.colab import files

In [0]:
%load_ext autoreload
%autoreload 1
%aimport graph
%aimport coarsening
%aimport utils

In [0]:
# Graphs.
number_edges = 8
metric ='euclidean'
normalized_laplacian = True
coarsening_levels = 4
len_img = 28

In [0]:
# Load dataset from local

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))
 

In [0]:
# Useful functions

def grid_graph(m):
  z = graph.grid(m)  # normalized nodes coordinates
  dist, idx = graph.distance_sklearn_metrics(z, k=number_edges, metric=metric) 
  #dist contains the distance of the 8 nearest neighbors for each node indicated in z sorted in ascending order
  #idx contains the indexes of the 8 nearest for each node sorted in ascending order by distance

  A = graph.adjacency(dist, idx)  # graph.adjacency() builds a sparse matrix out of the identified edges computing similarities as: A_{ij} = e^(-dist_{ij}^2/sigma^2)

  return A, z

  
def plot_matrix(m):
  plt.figure(figsize = (10, 10))
  plt.imshow(m.toarray())
  plt.show


# def convert_coo_to_sparse_tensor(L):
#   indices = np.column_stack((L.row, L.col))
#   L = tf.SparseTensor(indices, L.data.astype('float32'), L.shape)
#   L = tf.sparse.reorder(L)
#   return L

      
# def get_neighbour_indices(A, node_index):
#   indices = A.indices[tf.equal(A.indices[:,0], node_index)][:,1]
#   return indices

  
# def get_neighbour_features(X, neighbour_indices):
#   X = tf.constant(X)
#   features = tf.gather(X, neighbour_indices)
#   return features
  

def mask():
  space = np.arange(-1, 2)
  X, Y  = np.meshgrid(space,space)
  mask = np.array((X.ravel(), Y.ravel())).T
  mask = np.array([mask[:mask.shape[0]//2], mask[mask.shape[0]//2 + 1 :]])
  mask = mask.reshape((-1, 2))
#   plt.scatter(mask[:,0], mask[:,1])
#   plt.show()
#   mask = np.repeat(mask, (784)).reshape((784, 8, 2)).T
  return tf.constant(mask.astype(np.float32))


In [558]:
# Load MNIST dataset

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype(np.float32) / 255
x_test = x_test.astype(np.float32) / 255
y_train = y_train.astype(np.float32) 
y_test = y_test.astype(np.float32)

val_n = len(x_train)//100 * 15
(x_val, y_val) = x_train[0:val_n], y_train[0:val_n]
(x_train, y_train) = x_train[val_n:], y_train[val_n:]
x_val = x_val.astype(np.float32)
y_val = y_val.astype(np.float32)

# x_train = x_train.reshape(-1, len_img*len_img)
# x_val = x_val.reshape(-1, len_img*len_img)
# x_test = x_test.reshape(-1, len_img*len_img)
x_train = x_train.reshape(-1, len_img*len_img, 1)
x_val = x_val.reshape(-1, len_img*len_img, 1)
x_test = x_test.reshape(-1, len_img*len_img, 1)

print(x_train.shape)

(51000, 784, 1)


In [0]:
# Load adjacency matrix

# A, nodes_coordinates = grid_graph(len_img)
# A = convert_coo_to_sparse_tensor(A.tocoo())

# A_idx = tf.transpose(A.indices)

In [559]:
# Create adjacency matrix
l = x_train.shape[-2]

z = graph.grid(28)
dist, idx = graph.distance_sklearn_metrics(z, k=number_edges, metric=metric) 
cols = idx.ravel()
# We want 8 neighbours for each node (number_edges)
rows = np.arange(0,idx.shape[0]).repeat(number_edges)
idx = np.mat([rows, cols]).T
# Each edge has unitary weight
values = tf.ones(indices.shape[0])
A = tf.SparseTensor(indices=idx, values=values, dense_shape=(l,l))

print(A.shape)
print(A.indices)

(784, 784)
tf.Tensor(
[[  0   1]
 [  0  28]
 [  0  29]
 ...
 [783 726]
 [783 753]
 [783 725]], shape=(6272, 2), dtype=int64)


In [560]:
# Compute MNIST mask
u = mask()
# u = np.reshape(u, (8,2,1))
# u = np.reshape(u, (9,2,1))
print(u.shape)

(8, 2)


In [0]:
# Create Gaussian weightning 

class Weighting(Layer):

  def __init__(self, A, u, d, **kwargs):
    self.A = A
    self.u = u
    self.d = d
    super(Weighting, self).__init__(**kwargs)

  def build(self, input_shape):
    self.mu = self.add_weight(name='mu',
                              shape=(1, self.d),
                              initializer='uniform',
                              trainable=True)
    self.sigma = self.add_weight(name='sigma',
                                 shape=(1, self.d),
                                 initializer='uniform',
                                 trainable=True)
    super(Weighting, self).build(input_shape)  

  def call(self, X):
    diff =  (self.u - self.mu)**2
    factor = 1e-14 + self.sigma**2
    weights = -0.5 * (diff / factor)
#     weights = (8, 2)
    weights = tf.math.reduce_sum(weights, axis=-1)
#     weights = (8,1)
    weights = tf.math.softmax(weights, axis=0)
#     repeat it for 784 nodes
    weights = tf.ones([tf.shape(X)[0], weights.shape[0]]) * weights
#     weights = tf.reshape(weights, shape=[1,weights.shape[0]])
#     multiples = tf.constant(np.array([n_nodes, 1]))
#     weights = tf.tile(weights, multiples)
    weights = tf.SparseTensor(indices=self.A.indices,
                              values=tf.squeeze(tf.reshape(weights, (-1,1))), 
                              dense_shape=(self.A.dense_shape))
    
    neighbours_idxs = self.A.indices[:,1] 
    neighbours_features = tf.gather(X, self.A.indices[:,1], axis=-1)
    neighbours_features = tf.reshape(neighbours_features, (784, 8))
#     now that, for each node in the graph, I have the 8 neighbours' features, I can weight them with the gaussian weights already created
    weighted_neighbours_features = tf.sparse.sparse_dense_matmul(weights, neighbours_features)
#     obtained the 8 neighbours' weighted features for each node (784,8), I can sum up over the neighbours
    D = tf.reduce_sum(weighted_neighbours_features, axis=-1)
    
    return D
    
  def compute_output_shape(self, input_shape):
    return (input_shape[0], self.output_dim)

  
# Create Gaussian (kernel) layer

class MoNet(Layer):

  def __init__(self, A, u, d, n_gaussian, n_classes, **kwargs):
    self.A = A
    self.u = u
    self.d = d
    self.n_gaussian = n_gaussian
    self.n_classes = n_classes
    self.weightings = []
    for k in range(self.n_gaussian):
      weighting = Weighting(self.A, self.u, self.d)
      self.weightings.append(weighting)
    super(MoNet, self).__init__(**kwargs)

  def build(self, input_shape):
    self.W = self.add_weight(name='W', 
                             shape=(input_shape[-2], self.n_classes),
                             initializer='uniform',
                             trainable=True)

    super(MoNet, self).build(input_shape)  # Be sure to call this at the end

  def call(self, X):
    X = tf.squeeze(X)
    
    weightings = []
    for k in range(self.n_gaussian):
      weighting = self.weightings[k](X)
      weightings.append(weighting)
    
    h = tf.add_n(weightings)
#     print(h.shape)
    h = tf.reshape(h, shape=[1, tf.shape(h)[0]])
#     print(h.shape)
    h = h @ self.W
    h = tf.squeeze(h)

    return h

  def compute_output_shape(self, input_shape):
    return (input_shape[0], self.output_dim)

In [0]:
# class MoNet(Layer):

#   def __init__(self, A, u, d, n_gaussian, n_classes, batch_size, **kwargs):
#     self.A = A
#     self.u = u
#     self.d = d
#     self.n_gaussian = n_gaussian
#     self.n_classes = n_classes
#     self.batch_size = batch_size
#     self.weightings = []
#     for k in range(self.n_gaussian):
#       weighting = Weighting(self.A, self.u, self.d)
#       self.weightings.append(weighting)
#     super(MoNet, self).__init__(**kwargs)

#   def build(self, input_shape):
#     self.W = self.add_weight(name='W', 
#                              shape=(input_shape[-2], self.n_classes),
#                              initializer='uniform',
#                              trainable=True)

#     super(MoNet, self).build(input_shape)  # Be sure to call this at the end

#   def gaussian(self, X):
#     X = tf.squeeze(X)
    
#     weightings = []
#     for k in range(self.n_gaussian):
#       weighting = self.weightings[k](X)
#       weightings.append(weighting)
    
#     h = tf.add_n(weightings)
#     h = tf.reshape(h, shape=[1, tf.shape(h)[0]])
    
#     return h
    
#   def call(self, X):
#     if self.batch_size > 1:
#       h = tf.concat([self.gaussian(x) for x in X], axis=0)
#     else:
#       h = self.gaussian(X)
    
#     h = h @ self.W
#     h = tf.squeeze(h)
    
#     return h

#   def compute_output_shape(self, input_shape):
#     return (input_shape[0], self.output_dim)

In [0]:
# Hyperparameters

epochs=1
batch_size = 1
d = 2
n = 8
n_classes=10

In [0]:
# Build the model

model = Sequential()
model.add(MoNet(A=A, u=u, d=d, n_gaussian=5, n_classes=n_classes))
# model.add(Activation('softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy')

# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])


In [568]:
# Train the model
%%time
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=batch_size, shuffle=False)

Train on 51000 samples, validate on 9000 samples
51000/51000 [==============================] - 493s 10ms/sample - loss: 35.9171 - val_loss: 35.7811
CPU times: user 11min 4s, sys: 1min 29s, total: 12min 33s
Wall time: 8min 15s


In [545]:
# k = Weighting(A, u, d)
# a = k(x_train[0,:])
# print(a)
k = MoNet(A, u, d, 25, n_classes)
k(x_train[0,:,:])
# print(x_train[0:5,:,:].shape)

<tf.Tensor: id=503057, shape=(10,), dtype=float32, numpy=
array([-105.2822   ,  -45.20735  ,    0.5364599,   60.99408  ,
         33.609184 ,   -0.6513729,  -11.745907 , -114.55999  ,
        -84.71961  ,  -31.023523 ], dtype=float32)>